# Downloading the Videos in the VGKG Dataset Articles

In [25]:
from gzip import GzipFile
import pandas as pd
import glob
from multiprocessing.dummy import Pool as DummyPool

%load_ext autoreload
%autoreload 2

# Autoreload code so no jupyter restarts are required
from src.data import webpage 
from src import util 

data_dir = "../../../../data/"

In [11]:
data_count = 5000
with GzipFile(data_dir + '/external/vgkg-20160427-part1.csv.gz') as gzipfile:
    df = pd.read_csv(gzipfile, nrows=data_count)

# only request the data for pages where useful text could be extracted
files = glob.glob(data_dir + "/processed/sentences/[0-9]*")

686

In [23]:
urls = list()
for file in files:
    urls.append((file.split("/")[-1], df.iloc[file_index].DocumentIdentifier))
    
len(urls)

686

In [ ]:


f = FloatProgress(min=0, max=data_count)
display(f)
pool = DummyPool(16) # using dummypool here because for whatever reason the other one doesn't get proper bandwidth
results = list()



for result in pool.imap(webpage.download_and_save, ):
    f.value += 1
    results.append(result)
pool.close()
pool.join()

In [27]:
# Does Pool recognize it when the array is dynamically extended by the task?





TypeError: a bytes-like object is required, not 'str'

In [ ]:
from bs4 import BeautifulSoup
from __future__ import unicode_literals
import youtube_dl
ydl_opts = {
    'format': 'worstvideo', # bestvideo[height<=360]+bestaudio/best[height<=360]
    #geo-bypass: True
    #max-filesize: 200m
    #--audio-quality 9
}

from pytube import YouTube
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    def func(path):
        soup = BeautifulSoup(util.load_gzip_pickle(path))
        iframes = soup.findAll("iframe")
        for iframe in iframes:
            if iframe.has_attr("src"):
                src = iframe['src']
                if "youtube.com" in src:
                    try:
                         YouTube(iframe['src']).streams.filter(progressive=True, file_extension='mp4').order_by('resolution').asc().first().download(filename="Test")
                        #ydl.download([iframe['src']], info_filename="../" + path.split("/")[-1])
                    except Exception as e:
                        print(e)
   

    files = glob.glob(data_dir + "/raw/articles/[0-9]*")
    for file in files:
        func(file)

In [ ]:
# Check how many files contain an embedded youtube video
# <iframe width="560" height="315" src="https://www.youtube.com/embed/wogb2ctOfV4" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

In [ ]:
files = glob.glob(data_dir + "/raw/articles/[0-9]*")
f = FloatProgress(min=0, max=len(files))
display(f)
pool = Pool(20)
results = list()
for result in pool.imap(webpage.extract_sentences_and_save, files):
    f.value += 1
    results.append(result)
pool.close()
pool.join() 